# Using FastText via Gensim

This tutorial is about using the Gensim wrapper for the [FastText](https://github.com/facebookresearch/fastText) library for training FastText models, loading them and performing similarity operations and vector lookups analogous to Word2Vec.

## When to use FastText?
The main principle behind FastText is that the morphological structure of a word carries important information about the meaning of the word, which is not taken into account by traditional word embeddings, which train a unique word embedding for every individual word. This is especially significant for morphologically rich languages (German, Turkish) in which a single word can have a large number of morphological forms, each of which might occur rarely, thus making it hard to train good word embeddings.  
FastText attempts to solve this by treating each word as the aggregation of its subwords. For the sake of simplicity and language-independence, subwords are taken to the character ngrams of the word. The vector for a word is simply taken to be the sum of all vectors of its component char-ngrams.  
According to a detailed comparison of Word2Vec and FastText in [this notebook](https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/Word2Vec_FastText_Comparison.ipynb), FastText does significantly better on syntactic tasks as compared to the original Word2Vec, especially when the size of the training corpus is small. Word2Vec slightly outperforms FastText on semantic tasks though. The differences grow smaller as the size of training corpus increases.  
Training time for FastText is significantly higher than the Gensim version of Word2Vec (`15min 42s` vs `6min 42s` on text8, 17 mil tokens, 5 epochs, and a vector size of 100).  
FastText can be used to obtain vectors for out-of-vocabulary (oov) words, by summing up vectors for its component char-ngrams, provided at least one of the char-ngrams was present in the training data.

## Training models

For the following examples, we'll use the Lee Corpus (which you already have if you've installed gensim)

You need to have FastText setup locally to be able to train models. See [installation instructions for FastText](https://github.com/facebookresearch/fastText/#requirements) if you don't have FastText installed.

In [1]:
import gensim, os
from gensim.models.wrappers.fasttext import FastText

# Set FastText home to the path to the FastText executable
ft_home = '/home/jayant/Projects/fastText/fasttext'

# Set file names for train and test data
data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data']) + os.sep
lee_train_file = data_dir + 'lee_background.cor'

model = FastText.train(ft_home, lee_train_file)

print(model)

FastText(vocab=1762, size=100, alpha=0.025)


Hyperparameters for training the model follow the same pattern as Word2Vec. FastText supports the folllowing parameters from the original word2vec - 
     - model: Training architecture. Allowed values: `cbow`, `skipgram` (Default `cbow`)
     - size: Size of embeddings to be learnt (Default 100)
     - alpha: Initial learning rate (Default 0.025)
     - window: Context window size (Default 5)
     - min_count: Ignore words with number of occurrences below this (Default 5)
     - loss: Training objective. Allowed values: `ns`, `hs`, `softmax` (Default `ns`)
     - sample: Threshold for downsampling higher-frequency words (Default 0.001)
     - negative: Number of negative words to sample, for `ns` (Default 5)
     - iter: Number of epochs (Default 5)
     - sorted_vocab: Sort vocab by descending frequency (Default 1)
     - threads: Number of threads to use (Default 12)
     
In addition, FastText has two additional parameters - 
    - min_n: min length of char ngrams to be used (Default 3)
    - max_n: max length of char ngrams to be used for (Default 6)
These control the lengths of character ngrams that each word is broken down into while training and looking up embeddings. If `max_n` is set to 0, or to be lesser than `min_n`, no character ngrams are used, and the model effectively reduces to Word2Vec.

In [2]:
model = FastText.train(ft_home, lee_train_file, size=50, alpha=0.05, min_count=10)
print(model)

FastText(vocab=816, size=50, alpha=0.025)


Continuation of training with FastText models is not supported.

## Saving/loading models

Models can be saved and loaded via the `load` and `save` methods.

In [3]:
model.save('saved_fasttext_model')
loaded_model = FastText.load('saved_fasttext_model')
print(loaded_model)

FastText(vocab=816, size=50, alpha=0.025)


The `save_word2vec_method` causes the vectors for ngrams to be lost. As a result, a model loaded in this way will behave as a regular word2vec model.  


## Word vector lookup
FastText models support vector lookups for out-of-vocabulary words by summing up character ngrams belonging to the word.

In [4]:
print('night' in model.wv.vocab)
print('nights' in model.wv.vocab)
print(model['night'])
print(model['nights'])

True
False
[-0.47196999 -0.17528     0.19518    -0.31948     0.42835999  0.083281
 -0.15183     0.43415001  0.41251001 -0.10186    -0.54948997  0.12667
  0.14816    -0.065804   -0.21105    -0.42304999  0.011494    0.53068
 -0.57410997 -0.53930998 -0.33537999  0.16154     0.12377    -0.23537
 -0.14629    -0.34777001  0.27304     0.20597     0.12581     0.36671999
  0.32075     0.27351999 -0.13311    -0.04975    -0.52293003 -0.2766
  0.11863    -0.009231   -0.66074997  0.018031    0.57145     0.35547
  0.21588001  0.14431    -0.31865999  0.32027     0.55005002  0.19374999
  0.36609    -0.54184002]
[-0.4256132  -0.11521876  0.20166218 -0.34812452  0.30932881  0.02802653
 -0.18951961  0.4175721   0.41008326 -0.09026544 -0.50756483  0.07746826
  0.09458492  0.01440104 -0.17157355 -0.35189211  0.00103696  0.50923289
 -0.49944138 -0.38334864 -0.34287725  0.18023167  0.18014225 -0.22820314
 -0.08267317 -0.31241801  0.26023088  0.20673522  0.07008089  0.31678561
  0.31590793  0.16198126 -0.0928

The word vector lookup operation only works if atleast one of the component character ngrams is present in the training corpus. For example -

In [5]:
# Raises a KeyError since none of the character ngrams of the word `axe` are present in the training data
model['axe']

KeyError: 'all ngrams for word axe absent from model'

The `in` operation works slightly differently from the original word2vec. It tests whether a vector for the given word exists or not, not whether the word is present in the word vocabulary. To test whether a word is present in the training word vocabulary -

In [6]:
# Tests if word present in vocab
print("word" in model.wv.vocab)
# Tests if vector present for word
print("word" in model)

False
True


## Similarity operations

Similarity operations work the same way as word2vec. Out-of-vocabulary words can also be used, provided they have atleast one character ngram present in the training data.

In [7]:
print("nights" in model.wv.vocab)
print("night" in model.wv.vocab)
model.similarity("night", "nights")

False
True


0.97944545147919504

Syntactically similar words generally have high similarity in FastText models, since a large number of the component char-ngrams will be the same. As a result, FastText generally does better at syntactic tasks than Word2Vec. A detailed comparison is provided [here](https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/Word2Vec_FastText_Comparison.ipynb).

Other similarity operations -

In [8]:
# The example training corpus is a toy corpus, results are not expected to be good, for proof-of-concept only
model.most_similar("nights")

[(u'12', 0.9912641048431396),
 (u'across', 0.990070641040802),
 (u'few', 0.9840448498725891),
 (u'deaths', 0.9840392470359802),
 (u'parts', 0.9835165739059448),
 (u'One', 0.9833074808120728),
 (u'running', 0.9832631349563599),
 (u'2', 0.982011079788208),
 (u'victory', 0.9806963801383972),
 (u'each', 0.9789758920669556)]

In [9]:
model.n_similarity(['sushi', 'shop'], ['japanese', 'restaurant'])

0.97543218704680112

In [10]:
model.doesnt_match("breakfast cereal dinner lunch".split())

'lunch'

In [11]:
model.most_similar(positive=['baghdad', 'england'], negative=['london'])

[(u'against', 0.94775390625),
 (u'after', 0.923099935054779),
 (u'West', 0.910752534866333),
 (u'again', 0.903070867061615),
 (u'arrest', 0.8878517150878906),
 (u'suicide', 0.8750319480895996),
 (u'After', 0.8682445287704468),
 (u'innings', 0.859328031539917),
 (u'Test', 0.8542338609695435),
 (u'during', 0.852535605430603)]

In [13]:
model.accuracy(questions='questions-words.txt')

[{'correct': [], 'incorrect': [], 'section': u'capital-common-countries'},
 {'correct': [], 'incorrect': [], 'section': u'capital-world'},
 {'correct': [], 'incorrect': [], 'section': u'currency'},
 {'correct': [], 'incorrect': [], 'section': u'city-in-state'},
 {'correct': [],
  'incorrect': [(u'HE', u'SHE', u'HIS', u'HER'),
   (u'HIS', u'HER', u'HE', u'SHE')],
  'section': u'family'},
 {'correct': [], 'incorrect': [], 'section': u'gram1-adjective-to-adverb'},
 {'correct': [], 'incorrect': [], 'section': u'gram2-opposite'},
 {'correct': [], 'incorrect': [], 'section': u'gram3-comparative'},
 {'correct': [(u'BIG', u'BIGGEST', u'GOOD', u'BEST')],
  'incorrect': [(u'GOOD', u'BEST', u'BIG', u'BIGGEST')],
  'section': u'gram4-superlative'},
 {'correct': [(u'GO', u'GOING', u'SAY', u'SAYING'),
   (u'LOOK', u'LOOKING', u'SAY', u'SAYING'),
   (u'RUN', u'RUNNING', u'SAY', u'SAYING'),
   (u'SAY', u'SAYING', u'LOOK', u'LOOKING')],
  'incorrect': [(u'GO', u'GOING', u'LOOK', u'LOOKING'),
   (u'GO',

In [15]:
# Word Movers distance
sentence_obama = 'Obama speaks to the media in Illinois'.lower().split()
sentence_president = 'The president greets the press in Chicago'.lower().split()

# Remove their stopwords.
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
sentence_obama = [w for w in sentence_obama if w not in stopwords]
sentence_president = [w for w in sentence_president if w not in stopwords]

# Compute WMD.
distance = model.wmdistance(sentence_obama, sentence_president)
distance

0.7756597547632444